# Retail Data Example

Below is a demo applying automated feature engineering to a retail dataset

In [1]:
import featuretools as ft
import pandas as pd

## Prepare data

We load this data into from a CSV file hosted on Amazon S3. The origial dataset is available for download [here](http://archive.ics.uci.edu/ml/datasets/online+retail)

We then break the file up into several entities

* **item_purchases**: items in each invoice
* **items**: items and associated descriptions
* **invoices**: invoices placed 
* **customers**: customers who placed invoices

In [2]:
es = ft.EntitySet("retail")

es.entity_from_csv("item_purchases",
                   csv_path="s3://featuretools-static/uk_online_retail.csv",
                   index="item_purchase_id",
                   make_index=True,
                   time_index="InvoiceDate",
                   parse_date_cols=["InvoiceDate"])

es.normalize_entity(new_entity_id="items",
                    base_entity_id="item_purchases",
                    index="StockCode",
                    additional_variables=["Description"])

es.normalize_entity(new_entity_id="invoices",
                    base_entity_id="item_purchases",
                    index="InvoiceNo",
                    additional_variables=["CustomerID","Country"])

es.normalize_entity(new_entity_id="customers",
                    base_entity_id="invoices",
                    index="CustomerID",
                    additional_variables=["Country"])

## Run Deep Feature Synthesis

The input to DFS is a set of entities and a list of relationships (defined by our EntitySet) and the "target_entity" to calculate features for. We can supply "cutoff times" to specify that we want to calculate features one year after a customer's first invoice.

The ouput of DFS is a feature matrix and the corresponding list of feature defintions

In [3]:
cutoff_times = es["customers"].df[["CustomerID", "first_invoices_time"]].rename(columns={"CustomerID": "instance_id", "first_invoices_time": "time"})
cutoff_times["time"] = cutoff_times["time"] + pd.Timedelta("365 days")
cutoff_times.head(3)

,instance_id,time
CustomerID,,
17850.0,17850.0,2011-12-01 08:26:00
13047.0,13047.0,2011-12-01 08:34:00
12583.0,12583.0,2011-12-01 08:45:00


In [4]:
from featuretools.primitives import AvgTimeBetween, Mean, Sum, Count, Day

feature_matrix, features = ft.dfs(entityset=es, target_entity="customers",
                                  cutoff_time=cutoff_times.sample(100),
                                  agg_primitives=[AvgTimeBetween, Mean, Sum, Count],
                                  trans_primitives=[Day], max_depth=5, verbose=True)

calulate_feature_matrix: 100%|██████████| 101/101 [00:44<00:00,  2.89it/s]


In [5]:
feature_matrix.sample(3)

,DAY(first_invoices_time),Country,COUNT(invoices),AVG_TIME_BETWEEN(item_purchases),AVG_TIME_BETWEEN(invoices),MEAN(item_purchases.Quantity),COUNT(item_purchases),MEAN(item_purchases.UnitPrice),SUM(item_purchases.UnitPrice),SUM(item_purchases.Quantity),...,MEAN(invoices.MEAN(item_purchases.UnitPrice)),MEAN(invoices.AVG_TIME_BETWEEN(item_purchases)),MEAN(invoices.MEAN(item_purchases.Quantity)),MEAN(invoices.COUNT(item_purchases)),MEAN(item_purchases.items.AVG_TIME_BETWEEN(item_purchases)),MEAN(item_purchases.items.COUNT(item_purchases)),MEAN(item_purchases.items.MEAN(item_purchases.UnitPrice)),MEAN(item_purchases.items.MEAN(item_purchases.Quantity)),MEAN(item_purchases.items.SUM(item_purchases.Quantity)),MEAN(item_purchases.items.SUM(item_purchases.UnitPrice))
CustomerID,,,,,,,,,,,,,,,,,,,,,
15812.0,26,United Kingdom,5,453790.500000,4537905.0,22.487805,41,5.051707,207.12,922,...,3.819674,0.0,48.085556,8.2,197200.908471,610.219512,5.537463,11.880670,8302.219512,2995.527561
15353.0,7,United Kingdom,2,157496.129032,9764760.0,5.444444,63,3.312857,208.71,343,...,3.623382,0.0,7.536765,31.5,119518.228805,501.301587,3.891205,8.587544,4695.984127,2468.699206
17928.0,7,United Kingdom,2,788697.272727,17351340.0,6.086957,23,2.988696,68.74,140,...,3.156275,0.0,9.186275,11.5,146500.008131,371.695652,19.645460,10.163183,4084.217391,10311.004348


In [6]:
features

[<Feature: DAY(first_invoices_time)>,
 <Feature: Country>,
 <Feature: COUNT(invoices)>,
 <Feature: AVG_TIME_BETWEEN(item_purchases)>,
 <Feature: AVG_TIME_BETWEEN(invoices)>,
 <Feature: MEAN(item_purchases.Quantity)>,
 <Feature: COUNT(item_purchases)>,
 <Feature: MEAN(item_purchases.UnitPrice)>,
 <Feature: SUM(item_purchases.UnitPrice)>,
 <Feature: SUM(item_purchases.Quantity)>,
 <Feature: MEAN(invoices.SUM(item_purchases.UnitPrice))>,
 <Feature: MEAN(invoices.SUM(item_purchases.Quantity))>,
 <Feature: MEAN(invoices.MEAN(item_purchases.UnitPrice))>,
 <Feature: MEAN(invoices.AVG_TIME_BETWEEN(item_purchases))>,
 <Feature: MEAN(invoices.MEAN(item_purchases.Quantity))>,
 <Feature: MEAN(invoices.COUNT(item_purchases))>,
 <Feature: MEAN(item_purchases.items.AVG_TIME_BETWEEN(item_purchases))>,
 <Feature: MEAN(item_purchases.items.COUNT(item_purchases))>,
 <Feature: MEAN(item_purchases.items.MEAN(item_purchases.UnitPrice))>,
 <Feature: MEAN(item_purchases.items.MEAN(item_purchases.Quantity))>,
